In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tqdm import tqdm # linear algebra
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1098635.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1251087.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1268739.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1251741.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1013916.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3748381.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3784903.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/978455.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1086675.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3684539.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3904556.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/1119906.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3692506.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/923240.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3622018.jpg
/kaggle/input/hot-dog-not-hot-dog/train/hot_dog/3842701.jpg
/kaggle/input/hot-dog-not-hot-dog/train/ho

/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/388733.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/634232.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/762910.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/784362.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/207335.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/133245.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/679424.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/646412.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/343270.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/400228.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/307557.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/397696.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/564558.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/530316.jpg
/kaggle/input/hot-dog-not-hot-dog/seefood/test/hot_dog/697329.jpg
/kaggle/in

In [39]:
imageSize = 224
num_classes = 2

In [40]:
train_dir = '/kaggle/input/hot-dog-not-hot-dog/train/'
test_dir = '/kaggle/input/hot-dog-not-hot-dog/seefood/test/'

In [41]:
import cv2
from skimage.transform import resize


def get_data(dir_name):
    X = []
    y = []
    for folderName in os.listdir(dir_name):
        if not folderName.startswith('.'):
            for image_filename in tqdm(os.listdir(dir_name + folderName)):
                img_file = cv2.imread(dir_name + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(folderName)
    X = np.asarray(X)
    y = np.asarray(y)        
    return X,y

In [42]:
train_X,train_y = get_data(train_dir)

100%|██████████| 249/249 [00:05<00:00, 42.77it/s]


In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
le = LabelEncoder()
train_y = le.fit_transform(train_y)

In [45]:
from keras.utils.np_utils import to_categorical
train_y = to_categorical(train_y)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2)

In [47]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=13)
X_test, y_test = shuffle(X_test, y_test, random_state=13)

In [48]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [49]:
base_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(imageSize, imageSize,3))

In [50]:
num_classes = 2
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
        layer.trainable = False

rms_prop = optimizers.RMSprop(lr=0.0001)

model.compile(loss='binary_crossentropy', 
                  optimizer=rms_prop, 
                  metrics=['accuracy'])


callbacks_list = [EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [52]:
history = model.fit(X_train,y_train, epochs=10,validation_data=(X_test,y_test), verbose=1,callbacks = callbacks_list)

Epoch 1/10
13/13 [==============================] - 26s 2s/step - loss: 1.5413 - accuracy: 0.6005 - val_loss: 0.7732 - val_accuracy: 0.7300
Epoch 2/10
13/13 [==============================] - 25s 2s/step - loss: 0.4056 - accuracy: 0.8492 - val_loss: 0.4078 - val_accuracy: 0.8400
Epoch 3/10
13/13 [==============================] - 25s 2s/step - loss: 0.4212 - accuracy: 0.8618 - val_loss: 0.6828 - val_accuracy: 0.7900
Epoch 4/10
13/13 [==============================] - 25s 2s/step - loss: 0.2173 - accuracy: 0.9221 - val_loss: 0.4692 - val_accuracy: 0.8100
Epoch 5/10
13/13 [==============================] - 25s 2s/step - loss: 0.1072 - accuracy: 0.9598 - val_loss: 0.5520 - val_accuracy: 0.7900
Epoch 6/10
13/13 [==============================] - 25s 2s/step - loss: 0.2049 - accuracy: 0.9020 - val_loss: 0.7861 - val_accuracy: 0.7600
Epoch 7/10
13/13 [==============================] - 25s 2s/step - loss: 0.0944 - accuracy: 0.9648 - val_loss: 0.4097 - val_accuracy: 0.8400
Epoch 8/10
13/13 [==

In [53]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [54]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'model.tflite')

/kaggle/working/your_file_name